In [1]:
import re, string, unicodedata
import pandas as pd
import numpy as np
import gensim
import os
from gensim import corpora, models
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models
from pprint import pprint
import pickle
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models import CoherenceModel

C:\Users\clwhitfield\Anaconda3\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5
C:\Users\clwhitfield\Anaconda3\lib\site-packages\_pytest\mark\structures.py:443: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)
C:\Users\clwhitfield\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. 

In [2]:
os.chdir('../')

In [3]:
eth_list = ['blacks_pre', 'blacks_post']
cities_list = ['nyc','LosAngeles','houston', 'chicago','philadelphia','combined']

In [3]:
eth_list = ['asians_pre', 'asians_post','blacks_pre', 'blacks_post']
cities_list = ['nycThread']

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=True removes punctuations

In [5]:
def make_bigrams(texts,data_words):
    bigram = gensim.models.Phrases(data_words, min_count=1, threshold=30) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [6]:
def compute_coherence_values_gensim(dictionary, corpus, texts, limit, start=4, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    # Compute Coherence Score
    from gensim.models import CoherenceModel
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           #iterations = 100,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [7]:
def compute_coherence_values_mallet(mallet_path, dictionary, corpus, texts, limit, start=4, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    from gensim.models import CoherenceModel

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [8]:
def convertldaGenToldaMallet(mallet_model):
    model_gensim = gensim.models.ldamodel.LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha, eta=0,
    )
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [9]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [12]:
def getTopicModels(sName,c):
    output_dir = os.getcwd() + r'/Models/'

    df = pd.read_csv(os.getcwd() + r'/data/cleaned_data/%s_%s.csv' %(sName,c), encoding = 'utf-8')
    df.cleaned_comments = df.cleaned_comments.apply(str).astype('U').values


    df = df[df.cleaned_comments.apply(lambda x: len(x.split(' ')) > 3)]
    df = df.reset_index(drop=True)
    data = df.cleaned_comments.tolist()
    data1 = df.comments.tolist() #to place raw text in sorted topics data
    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    
    #trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    #trigram_mod = gensim.models.phrases.Phraser(trigram)


    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words,data_words)

    # Create Dictionary
    id2word = corpora.Dictionary(data_words_bigrams)

    # Create Corpus
    texts = data_words_bigrams

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    
    try: 
        os.makedirs(output_dir +r'Corpora/', exist_ok = True) 
        #print("Directory '%s' created successfully" %output_dir) 
    except OSError as error: 
            print("Directory '%s' can not be created")
    
    output_file = '/Corpora/%s_%s_corpus.p'%(sName,c)
    with open(r'%s%s'%(output_dir,output_file), 'wb') as fp:
        pickle.dump(corpus, fp)

    model_list, coherence_values = compute_coherence_values_gensim(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=4, limit=22, step=3)
    
    limit=22; start=4; step=3;
    x = range(start, limit, step)
    
    # Print the coherence scores  Create write to file
    try: 
        os.makedirs(output_dir +r'Gensim/Text/', exist_ok = True) 
        #print("Directory '%s' created successfully" %output_dir) 
    except OSError as error: 
            print("Directory '%s' can not be created")            
    myfile = open(r'%sGensim/Text/%s_%s_coherence.txt'%(output_dir, sName,c),'w+', encoding = 'utf-8')
    for m, cv in zip(x, coherence_values):
        cv = round(cv,4)
        myfile.writelines("Num Topics = %s has Coherence Value of %s"%(m, cv))
        myfile.writelines('\n')


    # Select the model and print the topics
    best_result_index = coherence_values.index(max(coherence_values))
    optimal_model = model_list[best_result_index]
    model_topics = optimal_model.show_topics(formatted=False)
    #print(optimal_model.print_topics(num_words=10))
    myfile.writelines(f'''The {x[best_result_index]} topics gives the highest coherence score \
    of {coherence_values[best_result_index]}''')

    myfile.close()
    output_file = '/Gensim/%s_%s_model_list.p'%(sName,c)
    with open(r'%s%s'%(output_dir,output_file), 'wb') as fp:
        pickle.dump(model_list, fp)
        
    try: 
        os.makedirs(output_dir +r'Gensim/%s/'%c, exist_ok = True) 
        #print("Directory '%s' created successfully" %output_dir) 
    except OSError as error: 
            print("Directory '%s' can not be created") 
    # Visualize the topics
    

    for m in model_list:
        pyLDAvis.enable_notebook()
        try:
            vis = pyLDAvis.gensim_models.prepare(m, corpus, id2word, sort_topics=True)
            pyLDAvis.save_html(vis, '%s/Gensim/%s/%s_%s_%s.html'%(output_dir,c,sName,c,model_list.index(m)))
        except:
                continue
    #vis
    del model_list

    ##BEGIN Mallet

    os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})
    mallet_path = r'C:/mallet-2.0.8/bin/mallet.bat' # update this path
    #ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

    # Can take a long time to run.
    model_list, coherence_values = compute_coherence_values_mallet(mallet_path, dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=4, limit=22, step=3)

    limit=22; start=4; step=3;
    x = range(start, limit, step)
    # Print the coherence scores  Create write to file
    try: 
        os.makedirs(output_dir+r'Mallet/Text/', exist_ok = True) 
        #print("Directory '%s' created successfully" %output_dir) 
    except OSError as error: 
            print("Directory '%s' can not be created")            
    myfile = open(r'%sMallet/Text/%s_%s_coherence.txt'%(output_dir, sName,c),'w+', encoding = 'utf-8')
    for m, cv in zip(x, coherence_values):
        cv = round(cv,4)
        myfile.writelines("Num Topics = %s has Coherence Value of %s"%(m, cv))
        myfile.writelines('\n')
    # Select the model and print the topics
    best_result_index = coherence_values.index(max(coherence_values))
    optimal_model = model_list[best_result_index]
    model_topics = optimal_model.show_topics(formatted=False)
    #print(optimal_model.print_topics(num_words=10))
    myfile.writelines(f'''The {x[best_result_index]} topics gives the highest coherence score \
    of {coherence_values[best_result_index]}''')

    myfile.close()
    output_file = '/Mallet/%s_%s_model_list.p'%(sName,c)
    with open(r'%s%s'%(output_dir,output_file), 'wb') as fp:
        pickle.dump(model_list, fp)
    #optimal_model = convertldaGenToldaMallet(optimal_model) #below - loop through optimal_model create pyLDAviz for each save to file

    # Visualize the topics
    try: 
        os.makedirs(output_dir +r'Mallet/%s/'%c, exist_ok = True) 
        #print("Directory '%s' created successfully" %output_dir) 
    except OSError as error: 
            print("Directory '%s' can not be created")
            
    pyLDAvis.enable_notebook()

    for m in model_list:
        try:
            vis = pyLDAvis.gensim_models.prepare(convertldaGenToldaMallet(m), corpus, id2word, sort_topics=True)
            pyLDAvis.save_html(vis, '%s/Mallet/%s/%s_%s_%s.html'%(output_dir,c,sName,c,model_list.index(m)))
        except:
            continue
    #vis
    del model_list



In [13]:
for i in eth_list:
    for j in cities_list:
        getTopicModels(i,j)

In [19]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show ###export df to file
df_dominant_topic.head(10)

NameError: name 'optimal_model' is not defined

In [35]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.5515,"asian, asians, anti, american, racist, care, t...",fair the labor movement accused asians unscrup...
1,1.0,0.6712,"people, asians, asian, virus, americans, china...",speculation that speculation fact fake news kx...
2,2.0,0.6579,"asians, asian, chinese, americans, fuck, point...",ah typical commie tactic deflect misdirect typ...
3,3.0,0.5554,"schools, school, kids, students, high, test, e...",a progressive assault selective high schools i...
4,4.0,0.2882,"asians, white, blacks, whites, jews, latinos, ...",poc people color check white box asians poc wi...


In [36]:
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.5515,"asian, asians, anti, american, racist, care, t...",fair the labor movement accused asians unscrup...
1,1.0,0.6712,"people, asians, asian, virus, americans, china...",speculation that speculation fact fake news kx...
2,2.0,0.6579,"asians, asian, chinese, americans, fuck, point...",ah typical commie tactic deflect misdirect typ...
3,3.0,0.5554,"schools, school, kids, students, high, test, e...",a progressive assault selective high schools i...
4,4.0,0.2882,"asians, white, blacks, whites, jews, latinos, ...",poc people color check white box asians poc wi...
5,5.0,0.6781,"racist, people, asians, nyc, blacks, discrimin...",fake news if youre spreading fake news acid at...
6,6.0,0.3894,"groups, poor, nyc, minority, poverty, communit...",unfortunately chinese population nyc the poore...
7,7.0,0.4056,"black, people, crime, community, violence, asi...",well far fewer asians country black people bla...
8,8.0,0.5245,"people, post, person, asians, literally, fucki...",like i said i grew policies china i vote i cit...
9,9.0,0.7109,"racism, asians, country, group, racial, racist...",i especially asians nyc wearing body cams we n...


In [30]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show ###create condition to drop rows if Num_Documents = NaN. Export to file
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,7.0,"schools, school, kids, students, education, hi...",296.0,0.0678
1.0,13.0,"asian, person, hate, hate_crimes, woman, attac...",188.0,0.0431
2.0,18.0,"gt, year, times, home, face, new_york, free, t...",314.0,0.0720
3.0,0.0,"live, lot, south, time, pretty, friends, area,...",224.0,0.0513
4.0,15.0,"asians, racist, talking, race, understand, cou...",239.0,0.0548
...,...,...,...,...
4359.0,8.0,"asians, chinese, nyc, african, minorities, tim...",NaN,NaN
4360.0,2.0,"asians, people, shit, fucking, yeah, literally...",NaN,NaN
4361.0,10.0,"virus, china, people, chinese, covid, coronavi...",NaN,NaN
4362.0,0.0,"live, lot, south, time, pretty, friends, area,...",NaN,NaN


In [33]:
df_dominant_topics.to_csv(os.getcwd() + r'\test.csv', index=False, encoding="utf-8")

In [37]:
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.5515,"asian, asians, anti, american, racist, care, t...",fair the labor movement accused asians unscrup...
1,1.0,0.6712,"people, asians, asian, virus, americans, china...",speculation that speculation fact fake news kx...
2,2.0,0.6579,"asians, asian, chinese, americans, fuck, point...",ah typical commie tactic deflect misdirect typ...
3,3.0,0.5554,"schools, school, kids, students, high, test, e...",a progressive assault selective high schools i...
4,4.0,0.2882,"asians, white, blacks, whites, jews, latinos, ...",poc people color check white box asians poc wi...
5,5.0,0.6781,"racist, people, asians, nyc, blacks, discrimin...",fake news if youre spreading fake news acid at...
6,6.0,0.3894,"groups, poor, nyc, minority, poverty, communit...",unfortunately chinese population nyc the poore...
7,7.0,0.4056,"black, people, crime, community, violence, asi...",well far fewer asians country black people bla...
8,8.0,0.5245,"people, post, person, asians, literally, fucki...",like i said i grew policies china i vote i cit...
9,9.0,0.7109,"racism, asians, country, group, racial, racist...",i especially asians nyc wearing body cams we n...
